<a href="https://colab.research.google.com/github/huganea/huganea.github.io/blob/master/Copy_of_Assignment_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this assignment we are going to work with data that was downloaded using using the [yfinance](https://pypi.org/project/yfinance/) library. It containst daily closing prices of all symbols.

In [1]:
from multiprocessing   import Pool
from tqdm import *
import pandas as pd

In [2]:
#Question 1: Explain what the code below is trying to do. Describe each step.
data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = set(data_clean['NASDAQ Symbol'].tolist())
data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/otherlisted.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = set(data_clean['NASDAQ Symbol'].tolist())


In [3]:
#Describe your answer below.
#Answer 1: The first line reads a file named nasdaqtrading.txt from http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt, it separates the elements in the txt file with using | to differentiate different elements.
#The second line cleans the data by isolating the Test Issue column with only tuples that contain N in that column. 
#The third line converts the data_clean list to include just the Nasdaq Symbol column. It converts the Nasdaq Symbol column into a list, then turns the list into a set. A set would have non-repeating values and will sort them in ascending order. 
#The fourth line reads a file named otherlisted.txt from http://www.nasdaqtrader.com/dynamic/SymDir/otherlisted.txt, it separates the elements in the txt file using | to differentiate between different elements. 
#The fifth line cleans the data by isolating the Test Issue column with only tuples that contain N in that column. 
#The sixth line converts the data_clean list to include just the Nasdaq Symbol column. It converts the Nasdaq Symbol column into a list, then turns the list into a set. A set would have non-repeating values and will sort them in ascending order.

In [4]:
!wget  -O stocks.parquet https://www.dropbox.com/s/gzpi950gfdz9dsk/stocks.parquet?dl=1 # lets download the data

--2022-01-27 19:48:16--  https://www.dropbox.com/s/gzpi950gfdz9dsk/stocks.parquet?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/gzpi950gfdz9dsk/stocks.parquet [following]
--2022-01-27 19:48:16--  https://www.dropbox.com/s/dl/gzpi950gfdz9dsk/stocks.parquet
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc931ae6eb182c682e6d01f2a74.dl.dropboxusercontent.com/cd/0/get/Bely-siPwllK9gOrW7tImj38MRJnlay7nSqnEcpLPoag8e6x_b1E1lrCDSMCAjrFafNypXDwFC7eb4CmrZduIeTgNX_dPsW_xHK-koa6pDh3AehFA-n0SGCrqza2MBeqXhZ6mXVzimqzn-aZqDEeewka/file?dl=1# [following]
--2022-01-27 19:48:16--  https://ucc931ae6eb182c682e6d01f2a74.dl.dropboxusercontent.com/cd/0/get/Bely-siPwllK9gOrW7tImj38MRJnlay7nSqnEcpLPoag8e6x_b1E1lrCDSMCAjrFafNypXDwFC7eb4CmrZ

In [5]:
!ls stock_closing_prices.parquet -l #downloaded file

ls: cannot access 'stock_closing_prices.parquet': No such file or directory


In [6]:
#Bonus question 1 - find on internet and write the command that will download all data from the symbols list. 
symlist = data_clean['NASDAQ Symbol'].tolist()
dict = {'symbols': symlist}
sym = pd.DataFrame(dict)
sym.to_csv('symbols.csv')

In [7]:
data=pd.read_parquet('stocks.parquet', columns=['date','adj_close','symbol']) # reduce the number of columns that will be read into the memory
data['date']=pd.to_datetime(data['date'])
symboldates=data.groupby('symbol',as_index=False).agg(date_min=pd.NamedAgg(column="date", aggfunc="min"),date_max=pd.NamedAgg(column="date", aggfunc="max"))

In [8]:
data.head(1) # Below shows a sample of the data.
#date - describes the day of the price
#open - the price at which stock opened
#high - what was the intraday high
#low - what is the intraday low
#close - what was the closing price
#adj_close - the price calculated after adjusting for all future stock splits
#dividends - what are the dividends that were paid on the day.
#stock_splits - if the stock was split
#symbol the symbol of the stock


,date,adj_close,symbol
0,1962-01-02,0.045278,HPQ


In [9]:
# Question 2 find top 20 best performer stocks - i.e. the stocks that provided the best return.
# if the stocks adj_close on the earliest date was X and the price on the 2021-01-01 is y and the number of days elapsed between these dates is Z.
# Then return = Y/X/Z
# Sort the stocks by this return and find the top 20

In [13]:
df=symboldates.merge(data.rename(columns={'adj_close':'start_adj_close'}),left_on=['symbol','date_min'],right_on=['symbol','date']).drop('date',axis=1)
df=df.merge(data.rename(columns={'adj_close':'end_adj_close'}),left_on=['symbol','date_max'],right_on=['symbol','date']).drop('date',axis=1)
df['num_days']= (df.date_max-df.date_min).dt.days
 
df['return']=df.apply(lambda row: None if row.start_adj_close ==0 or row.num_days==0 else row.end_adj_close/row.start_adj_close/row.num_days, axis=1)
df=df.sort_values('return', ascending=False)
df=df.head(20)
df
#z = data['adj_close'] - data[data['date']=='2021-01-01']
#ratio = data['adj_close']/data[data['date']=='2021-01-01'] / z
#ratio.head(1)

,symbol,date_min,date_max,start_adj_close,end_adj_close,num_days,return
7126,NVO,1981-04-30,2021-12-27,2.275794e-09,111.629997,14851,3.302876e+06
6925,NKE,1980-12-02,2021-12-27,3.775067e-06,167.580002,15000,2.959418e+03
2420,DCI,1980-03-17,2021-12-27,9.812100e-06,57.939999,15260,3.869564e+02
9747,TJX,1987-06-26,2021-12-27,2.857077e-05,74.290001,12603,2.063167e+02
10765,WTRG,1980-03-17,2021-12-27,4.768529e-04,52.549999,15260,7.221605e+00
9932,TTC,1980-03-17,2021-12-27,1.051490e-03,99.879997,15260,6.224708e+00
10164,USB,1973-05-03,2021-12-27,7.828808e-04,56.740002,17770,4.078555e+00
6292,MCD,1966-07-05,2021-12-27,3.923203e-03,268.239990,20264,3.374097e+00
7729,PII,1987-09-16,2021-12-27,3.345692e-03,107.000000,12521,2.554223e+00
4565,HD,1981-09-22,2021-12-27,1.823626e-02,404.089996,14706,1.506773e+00


In [11]:
#Question 3: Create a copy of this notebook called assignment-aws and follow the instructions in readme to run the notebook on AWS compute instance.
# Compare performance of running natively on colab and colab+AWS